**Dataset**

In [ ]:
import os
import pandas as pd
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares
import numpy as np

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [ ]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [ ]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [ ]:
ratings = ratings[['user_id', 'movie_id', 'count']]

In [ ]:
ratings

,user_id,movie_id,count
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4


In [ ]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


**EDA**

In [ ]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [ ]:
# rating에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [ ]:
# 가장 인기있는 영화 30개(인기순)
movie_data = pd.merge(ratings, movies)
movie_count = movie_data.groupby('title')['count'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

**Add ratings**

In [ ]:
my_favorite = ['Godfather, The (1972)' , 'Back to the Future (1985)' ,'Matrix, The (1999)' ,'Men in Black (1997)' ,'Jurassic Park (1993)']
favorite_movie_id = movies[movies['title'].isin(my_favorite)]
my_movie = pd.DataFrame({'user_id': [6041]*5, 'movie_id': favorite_movie_id['movie_id'], 'count':[5]*5})

if not ratings.isin({'user_id':[6041]})['user_id'].any():
    ratings = ratings.append(my_movie)
ratings.tail(10)

,user_id,movie_id,count
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4
1000208,6040,1097,4
476,6041,480,5
847,6041,858,5
1250,6041,1270,5
1539,6041,1580,5
2502,6041,2571,5


In [ ]:
favorite_movie_id

,movie_id,title,genre
476,480,Jurassic Park (1993),Action|Adventure|Sci-Fi
847,858,"Godfather, The (1972)",Action|Crime|Drama
1250,1270,Back to the Future (1985),Comedy|Sci-Fi
1539,1580,Men in Black (1997),Action|Adventure|Comedy|Sci-Fi
2502,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller


In [ ]:
movie_data = pd.merge(ratings, movies, on='movie_id')
movie_data

,user_id,movie_id,count,title,genre
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...
836478,5851,3607,5,One Little Indian (1973),Comedy|Drama|Western
836479,5854,3026,4,Slaughterhouse (1987),Horror
836480,5854,690,3,"Promise, The (Versprechen, Das) (1994)",Romance
836481,5938,2909,4,"Five Wives, Three Secretaries and Me (1998)",Documentary


**CSR matrix**

In [ ]:
ratings

,user_id,movie_id,count
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
476,6041,480,5
847,6041,858,5
1250,6041,1270,5
1539,6041,1580,5


In [ ]:
num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

In [ ]:
num_user

6040

In [ ]:
num_movie

3628

In [ ]:
ratings.user_id

0          1
1          1
2          1
3          1
4          1
        ... 
476     6041
847     6041
1250    6041
1539    6041
2502    6041
Name: user_id, Length: 836483, dtype: int64

In [ ]:
ratings.movie_id

0       1193
1        661
2        914
3       3408
4       2355
        ... 
476      480
847      858
1250    1270
1539    1580
2502    2571
Name: movie_id, Length: 836483, dtype: int64

In [ ]:
csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)))
csr_data

<6042x3953 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

**Train**

In [ ]:
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [ ]:
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [ ]:
csr_data_transpose = csr_data.T
csr_data_transpose

<3953x6042 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [ ]:
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

In [ ]:
my_vector, godfather_vector = als_model.user_factors[6041], als_model.item_factors[858]

In [ ]:
my_vector

array([ 1.9732334e-02,  8.6736715e-01,  6.7894101e-01, -1.6589388e-01,
        5.8600110e-01,  3.5912466e-01, -1.7356344e-01,  6.0234022e-01,
       -3.4860454e-02, -3.0902091e-01, -3.4623399e-01,  4.7773850e-01,
        4.5883515e-01, -7.7136266e-01,  1.0784352e-01,  9.7579831e-01,
       -3.7164298e-01,  1.5987715e-01, -5.5151820e-01, -3.6059108e-01,
       -3.9243662e-01, -1.5006079e-01,  3.6282670e-01, -7.0138597e-01,
       -1.9868657e-01, -6.6481924e-01, -2.1263942e-01,  2.3261066e-01,
        2.7849862e-02,  5.8278583e-02,  1.0614414e+00,  5.9316301e-01,
        1.0924668e-01,  2.0176413e-02, -1.6833335e-01, -2.2237408e-01,
       -5.9824604e-01, -3.8473368e-01,  2.0957792e-01, -8.3853042e-01,
        1.1144031e+00, -1.5163895e-01,  6.3220665e-02, -7.4571222e-01,
       -2.5926569e-02, -2.1731954e-04, -4.1099313e-01,  6.3863206e-01,
        1.8237099e-03, -8.2237929e-02,  6.0757422e-01,  7.4720585e-01,
       -1.2697841e-01, -3.7240887e-01,  3.7404296e-01, -3.2948214e-01,
      

In [ ]:
godfather_vector

array([ 6.4880680e-03, -1.0071150e-02, -6.2177246e-03, -5.8680126e-03,
        3.0305002e-02,  6.4120956e-02, -3.9364368e-05,  5.5472706e-02,
        2.0947309e-02, -1.3166106e-02, -2.3291776e-02, -2.0719608e-02,
        2.1185011e-02, -5.9626978e-03,  5.2425085e-04,  5.1235382e-02,
        9.7055119e-03,  3.3557232e-02,  3.7074264e-02,  5.0291210e-02,
        2.3632102e-02,  1.3193214e-02,  2.5712490e-02, -1.5891971e-02,
       -1.0052825e-02, -3.8314452e-03, -5.4857167e-03,  2.7143857e-02,
        5.5974396e-03,  1.3497976e-02,  1.4030961e-02,  5.1008570e-03,
        3.0427996e-02,  1.9120958e-02, -1.8038057e-02,  2.4053152e-03,
        1.0010246e-03, -9.6621551e-03, -2.3563029e-02, -4.0799100e-03,
        3.1573273e-02,  7.9759266e-03, -1.4248440e-02, -3.5925296e-03,
        4.4409066e-02, -1.6136250e-02, -2.1604639e-02,  1.8469216e-02,
        8.0593275e-03,  1.4244272e-02,  3.9399046e-02,  2.2625154e-02,
       -1.3370529e-02,  8.6152460e-03,  5.4355189e-03,  5.7076145e-02,
      

In [ ]:
np.dot(my_vector, godfather_vector)

0.61820847

In [ ]:
toystory_vector = als_model.item_factors[1]
np.dot(my_vector, toystory_vector)

0.16070537

**Similar Movie**

In [ ]:
favorite_movie = 'Godfather, The (1972)'
movie_id = movies[movies['title']=='Godfather, The (1972)']['movie_id']
similar_movie = als_model.similar_items(movie_id.values[0], N=15)
similar_movie

[(858, 1.0000001),
 (1221, 0.96376973),
 (2023, 0.5309487),
 (1953, 0.49192822),
 (923, 0.35498658),
 (2695, 0.34212857),
 (1213, 0.33592606),
 (624, 0.33157814),
 (912, 0.31745505),
 (3595, 0.30721763),
 (1787, 0.30708656),
 (111, 0.29941976),
 (1207, 0.29378197),
 (1387, 0.28012535),
 (2194, 0.27800813)]

In [ ]:
movies[movies['movie_id'].isin([s[0] for s in similar_movie])]

,movie_id,title,genre
109,111,Taxi Driver (1976),Drama|Thriller
619,624,Condition Red (1995),Action|Drama|Thriller
847,858,"Godfather, The (1972)",Action|Crime|Drama
900,912,Casablanca (1942),Drama|Romance|War
911,923,Citizen Kane (1941),Drama
1189,1207,To Kill a Mockingbird (1962),Drama
1195,1213,GoodFellas (1990),Crime|Drama
1203,1221,"Godfather: Part II, The (1974)",Action|Crime|Drama
1366,1387,Jaws (1975),Action|Horror
1728,1787,Paralyzing Fear: The Story of Polio in America...,Documentary


**Favorite Movie**

In [ ]:
user = 6041
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(589, 0.6565444),
 (1221, 0.5241904),
 (2916, 0.43456084),
 (260, 0.35347295),
 (1573, 0.31569466),
 (110, 0.3056903),
 (1527, 0.30177245),
 (1196, 0.29441684),
 (1210, 0.28158402),
 (1240, 0.27406263),
 (780, 0.27045923),
 (2791, 0.25477815),
 (1097, 0.24566084),
 (3175, 0.24299057),
 (457, 0.23872316),
 (2028, 0.23649827),
 (1198, 0.2322864),
 (1544, 0.2319798),
 (2023, 0.22917403),
 (2628, 0.21907125)]

In [ ]:
movies[movies['movie_id'].isin([m[0] for m in movie_recommended])]

,movie_id,title,genre
108,110,Braveheart (1995),Action|Drama|War
257,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
453,457,"Fugitive, The (1993)",Action|Thriller
585,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
770,780,Independence Day (ID4) (1996),Action|Sci-Fi|War
1081,1097,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi
1178,1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
1180,1198,Raiders of the Lost Ark (1981),Action|Adventure
1192,1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
1203,1221,"Godfather: Part II, The (1974)",Action|Crime|Drama


**회고**

좋아하는 영화로 다음의 영화들을 선택하였습니다.
Godfather, The
Back to the Future
Matrix
Men in Black
Jurassic Park

Godfather에 대한 선호도를 측정한 결과, 0.62 정도가 측정되었고, 좋아하는 영화 목록에 없는 toystory에 대한 선호도를 측정한 결과, 0.16 정도가 측정되었습니다.

Godfather와 유사한 영화를 추천받아봤는데, Godfather의 장르인 Action|Crime|Drama 와 비슷한 장르의 영화를 추천받았습니다.

좋아하는 영화추천에서는 대부분의 장르가 Action이었습니다. 초기에 목록에 넣은 영화들의 장르에 Action이 많이 포함되어 있어서 이런 결과를 얻은 것 같습니다.